In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from ml_metrics import rmse
import matplotlib.pyplot as plt
import pandas_profiling

In [27]:
raw_data_df1=pd.read_csv('/Users/lijjumathew/Code/SMU/MSDS-QTW/case_study1/data/train.csv')
raw_data_df2=pd.read_csv('/Users/lijjumathew/Code/SMU/MSDS-QTW/case_study1/data/unique_m.csv')
del raw_data_df1['critical_temp']
df1_cols = raw_data_df1.columns.values.tolist()
df2_cols = raw_data_df2.columns.values.tolist()
col_list = df1_cols + df2_cols
full_data_df = pd.concat([raw_data_df1,raw_data_df2], axis = 1, ignore_index=True)
full_data_df.columns = col_list

In [28]:
full_data_df.describe()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp
count,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,...,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.0,21263.0,21263.000000
mean,4.115224,87.557631,72.988310,71.290627,58.539916,1.165608,1.063884,115.601251,33.225218,44.391893,...,0.034108,0.020535,0.036663,0.047954,0.042461,0.201009,0.0,0.0,0.0,34.421219
std,1.439295,29.676497,33.490406,31.030272,36.651067,0.364930,0.401423,54.626887,26.967752,20.035430,...,0.307888,0.717975,0.205846,0.272298,0.274365,0.655927,0.0,0.0,0.0,34.254362
min,1.000000,6.941000,6.423452,5.320573,1.960849,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000210
25%,3.000000,72.458076,52.143839,58.041225,35.248990,0.966676,0.775363,78.512902,16.824174,32.890369,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.365000
50%,4.000000,84.922750,60.696571,66.361592,39.918385,1.199541,1.146783,122.906070,26.636008,45.123500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,20.000000
75%,5.000000,100.404410,86.103540,78.116681,73.113234,1.444537,1.359418,154.119320,38.356908,59.322812,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,63.000000
max,9.000000,208.980400,208.980400,208.980400,208.980400,1.983797,1.958203,207.972460,205.589910,101.019700,...,5.800000,64.000000,8.000000,7.000000,19.000000,14.000000,0.0,0.0,0.0,185.000000


In [16]:
# Taking long time
#profile = full_data_df.profile_report(title="Super Conductor Dataset")
#profile



### Fitting a Linear Regression model to full dataset

In [29]:
# Create training and testing sets (cross-validation not needed)
train_set = full_data_df.sample(frac=0.7, random_state=100)
test_set = full_data_df[~full_data_df.isin(train_set)].dropna()
print(train_set.shape[0])
print(test_set.shape[0])

14884
6379


In [30]:
train_set.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp,material
19769,3,90.076290,80.310454,83.973364,75.932202,1.026861,1.094568,80.21207,5.574640,34.993083,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0.69,La1Ni2Ge2
10404,3,66.003500,64.614175,63.785463,62.145996,1.063299,0.841095,40.28250,28.621675,17.945590,...,0.0,0.0,0.0,0.0,0.0,0,0,0,6.41,Zr1V1.85Fe0.15
1829,6,93.410967,59.853122,68.968336,36.231372,1.544257,1.455340,192.98100,25.498847,65.450357,...,0.0,0.0,0.0,0.0,2.0,0,0,0,89.70,Bi2Sr2Ca0.8Nd0.2Cu2O8.26
15434,2,148.077735,139.088915,147.793387,139.077880,0.691228,0.061919,18.34453,135.943915,9.172265,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0.62,Gd0.01La0.99
16179,3,89.071867,89.071867,88.398753,88.398753,1.091170,1.091170,26.14840,8.716133,10.782964,...,0.0,0.0,0.0,0.0,0.0,0,0,0,11.90,Zr1Ru1As1


In [31]:
list(train_set.columns)

['number_of_elements',
 'mean_atomic_mass',
 'wtd_mean_atomic_mass',
 'gmean_atomic_mass',
 'wtd_gmean_atomic_mass',
 'entropy_atomic_mass',
 'wtd_entropy_atomic_mass',
 'range_atomic_mass',
 'wtd_range_atomic_mass',
 'std_atomic_mass',
 'wtd_std_atomic_mass',
 'mean_fie',
 'wtd_mean_fie',
 'gmean_fie',
 'wtd_gmean_fie',
 'entropy_fie',
 'wtd_entropy_fie',
 'range_fie',
 'wtd_range_fie',
 'std_fie',
 'wtd_std_fie',
 'mean_atomic_radius',
 'wtd_mean_atomic_radius',
 'gmean_atomic_radius',
 'wtd_gmean_atomic_radius',
 'entropy_atomic_radius',
 'wtd_entropy_atomic_radius',
 'range_atomic_radius',
 'wtd_range_atomic_radius',
 'std_atomic_radius',
 'wtd_std_atomic_radius',
 'mean_Density',
 'wtd_mean_Density',
 'gmean_Density',
 'wtd_gmean_Density',
 'entropy_Density',
 'wtd_entropy_Density',
 'range_Density',
 'wtd_range_Density',
 'std_Density',
 'wtd_std_Density',
 'mean_ElectronAffinity',
 'wtd_mean_ElectronAffinity',
 'gmean_ElectronAffinity',
 'wtd_gmean_ElectronAffinity',
 'entropy_E

In [23]:
train_index = train_set.index.values.astype(int)
test_index = test_set.index.values.astype(int)
# Converting the training and testing datasets back to matrix-formats
X_train = train_set.iloc[:, 0:8].values # returns the data; excluding the target
Y_train = train_set.iloc[:, -1].values # returns the target-only
X_test = test_set.iloc[:, 0:8].values # ""
Y_test = test_set.iloc[:, -1].values # ""

In [25]:
;;;

array([[  3.        ,  90.07629   ,  80.310454  , ...,   1.0268612 ,
          1.09456776,  80.21207   ],
       [  3.        ,  66.0035    ,  64.614175  , ...,   1.063299  ,
          0.84109457,  40.2825    ],
       [  6.        ,  93.41096667,  59.85312215, ...,   1.54425724,
          1.45533958, 192.981     ],
       ...,
       [  5.        ,  89.33718   ,  54.3195736 , ...,   1.44453717,
          1.41657019, 124.90825   ],
       [  5.        ,  85.553919  ,  90.23441689, ...,   1.41166892,
          1.26217797, 135.9646    ],
       [  5.        ,  60.93397464,  71.45535504, ...,   1.34182239,
          1.19566148, 122.90607   ]])